In [1]:
import pandas as pd
import numpy as np

In [2]:
df_original = pd.read_csv("order_details_table.csv")
df = df_original.head(1500)

In [3]:
df.head()

,ID,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [4]:
df = df[['Weatherconditions', 'Road_traffic_density', 'Type_of_vehicle', 'multiple_deliveries','Time_taken(min)']]

In [5]:
dict_correspendance = {'conditions Sunny':6,'conditions Stormy':2, 
                       'conditions Sandstorms':1, 'conditions Fog':3,
                       'conditions Windy':4,'conditions Cloudy':5}
df['Weatherconditions']=df['Weatherconditions'].replace(dict_correspendance)

dict_correspendance = {'High':2,'Jam':1, 
                       'Low':4, 'Medium':3}
df['Road_traffic_density']=df['Road_traffic_density'].str.strip().replace(dict_correspendance)

dict_correspendance = { 'motorcycle' : 3 , 'scooter' : 2 , 'electric_scooter' : 1 } 
df['Type_of_vehicle'] = df['Type_of_vehicle'].str.strip().replace(dict_correspendance)
df

,Weatherconditions,Road_traffic_density,Type_of_vehicle,multiple_deliveries,Time_taken(min)
0,6,2,3,0,(min) 24
1,2,1,2,1,(min) 33
2,1,4,3,1,(min) 26
3,6,3,3,1,(min) 21
4,5,2,2,1,(min) 30
...,...,...,...,...,...
1495,1,1,3,3,(min) 45
1496,5,1,2,1,(min) 35
1497,2,1,2,0,(min) 22
1498,4,2,3,1,(min) 27


In [6]:
def trim(x):
    
    i = 0
    while (i < len(x) and (x[i] < '0' or x[i] > '9')):
        i += 1
    j = i
    while (j < len(x) and (x[j] >= '0' and x[j] <= '9')):
        j += 1
    x = x[i:j]
    return x
df['Time_taken(min)'] = df['Time_taken(min)'].apply(trim)

In [7]:
"""
def delete_none(x):
    
    j = 0
    x = x.strip()
    while (x != None and j < len(x) and (x[j] >= '0' and x[j] <= '9')):
        j += 1
    if (j != len(x)):
        x = None
    return x
"""
#df['multiple_deliveries'] = df['multiple_deliveries'].apply(delete_none)
df = df.apply(pd.to_numeric, errors='coerce')
df.dropna(inplace=True)
print(df.isnull().sum())
#df['multiple_deliveries'] = df['multiple_deliveries'].astype(int)

Weatherconditions       0
Road_traffic_density    0
Type_of_vehicle         0
multiple_deliveries     0
Time_taken(min)         0
dtype: int64


In [8]:
df.head(11)

,Weatherconditions,Road_traffic_density,Type_of_vehicle,multiple_deliveries,Time_taken(min)
0,6.0,2.0,3.0,0.0,24
1,2.0,1.0,2.0,1.0,33
2,1.0,4.0,3.0,1.0,26
3,6.0,3.0,3.0,1.0,21
4,5.0,2.0,2.0,1.0,30
5,5.0,1.0,3.0,1.0,26
6,3.0,1.0,2.0,1.0,40
7,5.0,3.0,3.0,1.0,32
8,2.0,1.0,3.0,1.0,34
9,3.0,1.0,3.0,3.0,46


In [9]:
ligne = df.iloc[1]
print(df.shape)
print(ligne.iloc[4])
df.loc[1, 'Time_taken(min)'] = None
df.loc[9, 'Time_taken(min)'] = None
df.loc[5, 'Time_taken(min)'] = None

(1440, 5)
33.0


In [10]:
df.head(11)
df.to_csv("Delivry_Data.csv", encoding="utf-8")

In [11]:
def predict_time(ligne_index, df):
 
    nbr_rows= df.shape[0]
    distance = {}
    i = 0
    while (i < nbr_rows):
        if (pd.isna(df.iloc[i]['Time_taken(min)'])):
            distance[i] = np.inf
        else:
            distance[i] = np.sqrt(np.sum((df.iloc[ligne_index][:-1] - df.iloc[i][:-1]) ** 2))
        i += 1
    distance = dict(sorted(distance.items(), key=lambda item: item[1]))
    #print(distance)
    sum = 0
    k = int(np.sqrt(nbr_rows))
    i = 0
    while (i < k):
        key = list(distance.keys())[i]
        sum += df.iloc[key]['Time_taken(min)']
        i += 1
    return sum / k

In [12]:
print(predict_time(1439, df))


20.027027027027028


In [13]:
i = 0
while (i < 20):
    print("Valeur Origine: ",df.iloc[i]['Time_taken(min)']," Valeur Approchee: ",predict_time(i, df))
    i += 1

Valeur Origine:  24.0  Valeur Approchee:  22.324324324324323
Valeur Origine:  nan  Valeur Approchee:  31.56756756756757
Valeur Origine:  26.0  Valeur Approchee:  21.513513513513512
Valeur Origine:  21.0  Valeur Approchee:  24.027027027027028
Valeur Origine:  30.0  Valeur Approchee:  30.027027027027028
Valeur Origine:  nan  Valeur Approchee:  33.86486486486486
Valeur Origine:  40.0  Valeur Approchee:  35.2972972972973
Valeur Origine:  32.0  Valeur Approchee:  28.08108108108108
Valeur Origine:  34.0  Valeur Approchee:  34.729729729729726
Valeur Origine:  nan  Valeur Approchee:  41.67567567567568
Valeur Origine:  23.0  Valeur Approchee:  28.08108108108108
Valeur Origine:  21.0  Valeur Approchee:  25.37837837837838
Valeur Origine:  20.0  Valeur Approchee:  20.7027027027027
Valeur Origine:  41.0  Valeur Approchee:  35.2972972972973
Valeur Origine:  20.0  Valeur Approchee:  24.16216216216216
Valeur Origine:  33.0  Valeur Approchee:  28.18918918918919
Valeur Origine:  40.0  Valeur Approchee: 

In [14]:
for i in range(20):
    if (pd.isna(df.iloc[i]['Time_taken(min)'])):
        df.loc[i, 'Time_taken(min)'] = predict_time(i, df)

In [15]:
df.head(11)

,Weatherconditions,Road_traffic_density,Type_of_vehicle,multiple_deliveries,Time_taken(min)
0,6.0,2.0,3.0,0.0,24.000000
1,2.0,1.0,2.0,1.0,31.567568
2,1.0,4.0,3.0,1.0,26.000000
3,6.0,3.0,3.0,1.0,21.000000
4,5.0,2.0,2.0,1.0,30.000000
5,5.0,1.0,3.0,1.0,33.864865
6,3.0,1.0,2.0,1.0,40.000000
7,5.0,3.0,3.0,1.0,32.000000
8,2.0,1.0,3.0,1.0,34.000000
9,3.0,1.0,3.0,3.0,41.675676
